# Using low text processing

In [2]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
nlp = spacy.load('en')

#import other libraries
import numpy as np
import pandas as pd
import random
import sys
import os
import time
import collections
from six.moves import cPickle

Using TensorFlow backend.


In [3]:
df_hack = pd.read_csv('input/HackerNoon.csv')
df_Ent = pd.read_csv('input/output.csv')
df_combined = pd.concat([df_hack, df_Ent])
df_combined.dropna(inplace = True)
df_combined.reset_index(inplace = True)

In [4]:
df_combined.head()

,index,Author,Site,Text
0,0,David Smooke,https://hackernoon.com/,“Google Cloud Platform and Firebase give Hacke...
1,1,David Smooke,https://hackernoon.com/,“Our serverless infrastructure will generate ...
2,2,David Smooke,https://hackernoon.com/,We’re excited to focus on important product d...
3,3,David Smooke,https://hackernoon.com/,”
4,4,David Smooke,https://hackernoon.com/,As a startup working to free ourselves from pl...


In [5]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

In [6]:
data_list = df_combined['Text']
wordlist = []
for paragraph in data_list.values:
    doc = nlp(paragraph)
    wl = create_wordlist(doc)
    wordlist += wl

In [7]:
word_counts = collections.Counter(wordlist)

# mapping index to word : vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))
# print (vocabulary_inv)

# Mapping from word to index
vocab = {x : i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

# print (vocab)
vocab_size = len(words)
print ("Vocab Size is  : ", vocab_size)

Vocab Size is  :  3030


In [8]:
# Creating sequences and next word for training.
SEQ_LENGTH = 30
SEQ_STEP = 1
sequences = []
next_words = []
for i in range(0, len(wordlist) - SEQ_LENGTH, SEQ_STEP):
    sequences.append(wordlist[i: i+SEQ_LENGTH])
    next_words.append(wordlist[i+SEQ_LENGTH])

print('nb sequences : ', len(sequences))

nb sequences :  15550


In [14]:
# creating matrix for vocab and mapping the words
X = np.zeros((len(sequences), SEQ_LENGTH, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype = np.bool)

In [15]:
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

In [16]:
def bidirectional_lstm_model(SEQ_LENGTH, vocab_size):
    RNN_SIZE = 256
    
    print('Building LSTM Model')
    model = Sequential()
    model.add(Bidirectional(LSTM(RNN_SIZE, activation='relu'), input_shape = (SEQ_LENGTH, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr = 0.001)
    callbacks = [EarlyStopping(patience = 2, monitor = 'val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
    print("model built!")
    return model

In [17]:
md = bidirectional_lstm_model(SEQ_LENGTH, vocab_size)
md.summary()

Building LSTM Model
model built!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               6731776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3030)              1554390   
_________________________________________________________________
activation_1 (Activation)    (None, 3030)              0         
Total params: 8,286,166
Trainable params: 8,286,166
Non-trainable params: 0
_________________________________________________________________


In [20]:
BATCH_SIZE = 32
EPOCHS = 50
save_dir = 'output'

callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=2, mode='auto', period=2)]
#fit the model
history = md.fit(X, y,
                 batch_size=BATCH_SIZE,
                 shuffle=True,
                 epochs=EPOCHS,
                 callbacks=callbacks,
                 validation_split=0.1)

#save the model
md.save(save_dir + "/" + 'my_model_generate_sentences.h5')

Train on 13995 samples, validate on 1555 samples
Epoch 1/50


 4640/13995 [========>.....................] - ETA: 7:46 - loss: 6.8917 - categorical_accuracy: 0.06 - ETA: 7:44 - loss: 6.7342 - categorical_accuracy: 0.06 - ETA: 7:40 - loss: 6.4638 - categorical_accuracy: 0.06 - ETA: 7:34 - loss: 6.4884 - categorical_accuracy: 0.05 - ETA: 7:34 - loss: 6.5568 - categorical_accuracy: 0.05 - ETA: 7:32 - loss: 6.6503 - categorical_accuracy: 0.04 - ETA: 7:37 - loss: 6.6456 - categorical_accuracy: 0.04 - ETA: 7:49 - loss: 6.6048 - categorical_accuracy: 0.04 - ETA: 8:00 - loss: 6.5406 - categorical_accuracy: 0.04 - ETA: 8:03 - loss: 6.4994 - categorical_accuracy: 0.04 - ETA: 8:00 - loss: 6.4925 - categorical_accuracy: 0.04 - ETA: 7:59 - loss: 6.4871 - categorical_accuracy: 0.04 - ETA: 7:56 - loss: 6.5051 - categorical_accuracy: 0.04 - ETA: 7:59 - loss: 6.5163 - categorical_accuracy: 0.04 - ETA: 8:01 - loss: 6.4824 - categorical_accuracy: 0.04 - ETA: 8:01 - loss: 6.4777 - categorical_accuracy: 0.04 - ETA: 8:04 - loss: 6.4532 - categorical_accuracy: 0.03 - E

13984/13995 [============================>.] - ETA: 2:41 - loss: 6.2966 - categorical_accuracy: 0.04 - ETA: 2:40 - loss: 6.2968 - categorical_accuracy: 0.04 - ETA: 2:39 - loss: 6.2973 - categorical_accuracy: 0.04 - ETA: 2:38 - loss: 6.2971 - categorical_accuracy: 0.04 - ETA: 2:37 - loss: 6.2975 - categorical_accuracy: 0.04 - ETA: 2:36 - loss: 6.2970 - categorical_accuracy: 0.04 - ETA: 2:35 - loss: 6.2969 - categorical_accuracy: 0.04 - ETA: 2:34 - loss: 6.2957 - categorical_accuracy: 0.04 - ETA: 2:33 - loss: 6.2965 - categorical_accuracy: 0.04 - ETA: 2:32 - loss: 6.2963 - categorical_accuracy: 0.04 - ETA: 2:31 - loss: 6.2963 - categorical_accuracy: 0.04 - ETA: 2:30 - loss: 6.2982 - categorical_accuracy: 0.04 - ETA: 2:29 - loss: 6.2972 - categorical_accuracy: 0.04 - ETA: 2:28 - loss: 6.2984 - categorical_accuracy: 0.04 - ETA: 2:27 - loss: 6.2963 - categorical_accuracy: 0.04 - ETA: 2:26 - loss: 6.2977 - categorical_accuracy: 0.04 - ETA: 2:24 - loss: 6.2983 - categorical_accuracy: 0.04 - E

13995/13995 [==============================] - 505s 36ms/step - loss: 6.2972 - categorical_accuracy: 0.0427 - val_loss: 6.6330 - val_categorical_accuracy: 0.0386
Epoch 2/50


 4672/13995 [=========>....................] - ETA: 7:56 - loss: 5.8758 - categorical_accuracy: 0.06 - ETA: 7:58 - loss: 6.0692 - categorical_accuracy: 0.04 - ETA: 7:44 - loss: 6.1900 - categorical_accuracy: 0.03 - ETA: 7:47 - loss: 6.2122 - categorical_accuracy: 0.03 - ETA: 8:01 - loss: 6.2960 - categorical_accuracy: 0.03 - ETA: 8:10 - loss: 6.3153 - categorical_accuracy: 0.03 - ETA: 8:08 - loss: 6.2727 - categorical_accuracy: 0.03 - ETA: 8:12 - loss: 6.3142 - categorical_accuracy: 0.04 - ETA: 8:18 - loss: 6.2734 - categorical_accuracy: 0.05 - ETA: 8:16 - loss: 6.2544 - categorical_accuracy: 0.05 - ETA: 8:15 - loss: 6.2798 - categorical_accuracy: 0.05 - ETA: 8:14 - loss: 6.2131 - categorical_accuracy: 0.05 - ETA: 8:12 - loss: 6.1801 - categorical_accuracy: 0.06 - ETA: 8:10 - loss: 6.1554 - categorical_accuracy: 0.05 - ETA: 8:07 - loss: 6.1474 - categorical_accuracy: 0.06 - ETA: 8:07 - loss: 6.1566 - categorical_accuracy: 0.05 - ETA: 8:05 - loss: 6.1828 - categorical_accuracy: 0.05 - E

...

 4672/13995 [=========>....................] - ETA: 6:54 - loss: 5.2436 - categorical_accuracy: 0.09 - ETA: 7:07 - loss: 5.4077 - categorical_accuracy: 0.09 - ETA: 7:06 - loss: 5.3799 - categorical_accuracy: 0.09 - ETA: 7:12 - loss: 5.5327 - categorical_accuracy: 0.08 - ETA: 7:11 - loss: 5.6687 - categorical_accuracy: 0.09 - ETA: 7:10 - loss: 5.6699 - categorical_accuracy: 0.07 - ETA: 7:11 - loss: 5.6969 - categorical_accuracy: 0.07 - ETA: 7:10 - loss: 5.7507 - categorical_accuracy: 0.07 - ETA: 7:09 - loss: 5.7148 - categorical_accuracy: 0.06 - ETA: 7:08 - loss: 5.6764 - categorical_accuracy: 0.07 - ETA: 7:06 - loss: 5.6789 - categorical_accuracy: 0.06 - ETA: 7:06 - loss: 5.6831 - categorical_accuracy: 0.06 - ETA: 7:06 - loss: 5.6717 - categorical_accuracy: 0.06 - ETA: 7:07 - loss: 5.7087 - categorical_accuracy: 0.06 - ETA: 7:07 - loss: 5.7363 - categorical_accuracy: 0.05 - ETA: 7:06 - loss: 5.6915 - categorical_accuracy: 0.06 - ETA: 7:04 - loss: 5.6865 - categorical_accuracy: 0.06 - E

13995/13995 [==============================] - ETA: 2:25 - loss: 5.6759 - categorical_accuracy: 0.06 - ETA: 2:24 - loss: 5.6778 - categorical_accuracy: 0.06 - ETA: 2:23 - loss: 5.6775 - categorical_accuracy: 0.06 - ETA: 2:22 - loss: 5.6774 - categorical_accuracy: 0.06 - ETA: 2:21 - loss: 5.6763 - categorical_accuracy: 0.06 - ETA: 2:20 - loss: 5.6771 - categorical_accuracy: 0.06 - ETA: 2:19 - loss: 5.6770 - categorical_accuracy: 0.06 - ETA: 2:18 - loss: 5.6784 - categorical_accuracy: 0.06 - ETA: 2:17 - loss: 5.6770 - categorical_accuracy: 0.06 - ETA: 2:16 - loss: 5.6778 - categorical_accuracy: 0.06 - ETA: 2:15 - loss: 5.6788 - categorical_accuracy: 0.06 - ETA: 2:14 - loss: 5.6773 - categorical_accuracy: 0.06 - ETA: 2:13 - loss: 5.6775 - categorical_accuracy: 0.06 - ETA: 2:12 - loss: 5.6769 - categorical_accuracy: 0.06 - ETA: 2:11 - loss: 5.6782 - categorical_accuracy: 0.06 - ETA: 2:10 - loss: 5.6763 - categorical_accuracy: 0.06 - ETA: 2:09 - loss: 5.6769 - categorical_accuracy: 0.06 - E


Epoch 00004: saving model to output/my_model_gen_sentences.04-6.81.hdf5
Epoch 5/50


 4608/13995 [========>.....................] - ETA: 6:40 - loss: 6.3297 - categorical_accuracy: 0.03 - ETA: 6:58 - loss: 5.9862 - categorical_accuracy: 0.03 - ETA: 7:04 - loss: 5.8900 - categorical_accuracy: 0.05 - ETA: 7:02 - loss: 5.8082 - categorical_accuracy: 0.04 - ETA: 7:03 - loss: 5.7883 - categorical_accuracy: 0.05 - ETA: 7:03 - loss: 5.7595 - categorical_accuracy: 0.05 - ETA: 7:04 - loss: 5.7552 - categorical_accuracy: 0.04 - ETA: 7:03 - loss: 5.7683 - categorical_accuracy: 0.05 - ETA: 7:01 - loss: 5.6223 - categorical_accuracy: 0.07 - ETA: 7:01 - loss: 5.5892 - categorical_accuracy: 0.08 - ETA: 7:00 - loss: 5.5046 - categorical_accuracy: 0.10 - ETA: 7:00 - loss: 5.5015 - categorical_accuracy: 0.09 - ETA: 6:59 - loss: 5.5179 - categorical_accuracy: 0.08 - ETA: 6:58 - loss: 5.5010 - categorical_accuracy: 0.09 - ETA: 6:58 - loss: 5.5339 - categorical_accuracy: 0.08 - ETA: 6:57 - loss: 5.4670 - categorical_accuracy: 0.08 - ETA: 6:56 - loss: 5.4534 - categorical_accuracy: 0.09 - E

13995/13995 [==============================] - ETA: 0s - loss: 5.4132 - categorical_accuracy: 0.08 - 466s 33ms/step - loss: 5.4130 - categorical_accuracy: 0.0877 - val_loss: 6.9203 - val_categorical_accuracy: 0.0598


In [22]:
# model trained, now saving vocabulary
vocab_file = os.path.join('output', "words_vocab.pkl")
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

In [24]:
from keras.models import load_model
print ('loading vocabulary...')
vocab_file = os.path.join('output', 'words_vocab.pkl')

with open(os.path.join('output', 'words_vocab.pkl'), 'rb') as f:
    words, vocab, voabulary = cPickle.load(f)

vocab_size = len(words)

print ('loading the model...')
model = load_model('output' + "/" + 'my_model_generate_sentences.h5')
print (model.summary())

def sample(preds, temperature=0.5):

    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

word_numbers = 30
seed_sentence = 'i learn that google is good for you , but is this following product good enough for you ?'

generated = ''
sentence = ['a']*SEQ_LENGTH

seed = seed_sentence.split()


for i in range(len(seed)):
    sentence[SEQ_LENGTH - i -1] = seed[len(seed) - i - 1]
# print(sentence)
generated += ' '.join(sentence)

for i in range(word_numbers):
    x = np.zeros((1, SEQ_LENGTH, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1

    preds = model.predict(x, verbose = 0)[0]
    next_index = sample(preds, 0.33)
    next_word = vocabulary_inv[next_index]

    generated += ' ' + next_word
    sentence = sentence[1:] + [next_word]

print (generated)

loading vocabulary...
loading the model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               6731776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3030)              1554390   
_________________________________________________________________
activation_1 (Activation)    (None, 3030)              0         
Total params: 8,286,166
Trainable params: 8,286,166
Non-trainable params: 0
_________________________________________________________________
None
a a a a a a a a a a a i learn that google is good for you , but is this following product good enough for you ? the same same home , you the ai market the new business the model   the network the file   a product you